In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from datetime import datetime, timedelta

from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import leaguegamefinder, playernextngames, commonallplayers

# NBA API (nba_api)

## search schedule by team (won't load future games)

In [2]:
# Function to get the team ID by team name
def get_team_id(team_name):
    nba_teams = teams.get_teams()
    for team in nba_teams:
        if team['full_name'] == team_name:
            return team['id']
    return None

In [4]:
# Function to get the upcoming schedule for a team
def get_team_schedule(team_id):
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
    games = gamefinder.get_data_frames()[0]
    upcoming_games = games[games.SEASON_ID.str[-4:] == '2023'] # games[games['SEASON_ID'] == '22023']  # Replace '22023' with the desired season ID
    return upcoming_games[['GAME_DATE', 'MATCHUP']]

In [3]:
# Replace 'Los Angeles Lakers' with the name of the desired NBA team
team_name = 'Denver Nuggets'
team_id = get_team_id(team_name)

In [5]:
schedule = get_team_schedule(team_id)
schedule

,GAME_DATE,MATCHUP
0,2024-04-14,DEN @ MEM
1,2024-04-12,DEN @ SAS
2,2024-04-10,DEN vs. MIN
3,2024-04-09,DEN @ UTA
4,2024-04-06,DEN vs. ATL
...,...,...
87,2023-07-15,DEN @ NYK
88,2023-07-14,DEN vs. MIA
89,2023-07-12,DEN @ UTA
90,2023-07-09,DEN vs. ATL


In [100]:
commonallplayers.CommonAllPlayers(is_only_current_season=1).get_data_frames()[0]

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,PLAYER_SLUG,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_SLUG,TEAM_CODE,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
0,1630173,"Achiuwa, Precious",Precious Achiuwa,1,2020,2023,precious_achiuwa,precious_achiuwa,1610612752,New York,Knicks,NYK,knicks,knicks,Y,00
1,203500,"Adams, Steven",Steven Adams,1,2013,2023,steven_adams,steven_adams,1610612745,Houston,Rockets,HOU,rockets,rockets,Y,00
2,1628389,"Adebayo, Bam",Bam Adebayo,1,2017,2023,bam_adebayo,bam_adebayo,1610612748,Miami,Heat,MIA,heat,heat,Y,00
3,1630534,"Agbaji, Ochai",Ochai Agbaji,1,2022,2023,ochai_agbaji,ochai_agbaji,1610612761,Toronto,Raptors,TOR,raptors,raptors,Y,00
4,1630583,"Aldama, Santi",Santi Aldama,1,2021,2023,santi_aldama,santi_aldama,1610612763,Memphis,Grizzlies,MEM,grizzlies,grizzlies,Y,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,201152,"Young, Thaddeus",Thaddeus Young,1,2007,2023,thaddeus_young,thaddeus_young,1610612756,Phoenix,Suns,PHX,suns,suns,Y,00
573,1629027,"Young, Trae",Trae Young,1,2018,2023,trae_young,trae_young,1610612737,Atlanta,Hawks,ATL,hawks,hawks,Y,00
574,1630209,"Yurtseven, Omer",Omer Yurtseven,1,2020,2023,omer_yurtseven,omer_yurtseven,1610612762,Utah,Jazz,UTA,jazz,jazz,Y,01
575,203469,"Zeller, Cody",Cody Zeller,1,2013,2023,cody_zeller,cody_zeller,1610612740,New Orleans,Pelicans,NOP,pelicans,pelicans,Y,00


In [108]:
commonallplayers.CommonAllPlayers(is_only_current_season=1).get_data_frames()[0].DISPLAY_FIRST_LAST.iloc[0]

'Precious Achiuwa'

## search schedule by player

In [9]:
all_players = players.get_players()
player_name = "Nikola Jokic"
jokic = [player for player in all_players if player['full_name'] == player_name]
jokic_id = jokic[0]['id']

In [10]:
def get_next_n_games_for_player(player_id, num_games=5):
    player_next_n_games = playernextngames.PlayerNextNGames(player_id=player_id)
    player_next_n_games_df = player_next_n_games.get_data_frames()[0]
    next_n_games = player_next_n_games_df.head(num_games)
    return next_n_games

In [11]:
df_nuggets_5 = get_next_n_games_for_player(jokic_id)
df_nuggets_5

,GAME_ID,GAME_DATE,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_NAME,VISITOR_TEAM_NAME,HOME_TEAM_ABBREVIATION,VISITOR_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_NICKNAME,GAME_TIME,HOME_WL,VISITOR_WL


In [81]:
# Function to subtract two hours from the time string
def subtract_two_hours(time_str):
    # Convert string to datetime object
    datetime_obj = datetime.strptime(time_str, '%I:%M %p')
    
    # Subtract two hours
    datetime_obj -= timedelta(hours=2)
    
    # Format the datetime object as a string in '%I:%M %p' format
    return datetime_obj.strftime('%I:%M %p')

In [84]:
df_nuggets_display = df_nuggets_5[['GAME_DATE','HOME_TEAM_ABBREVIATION','VISITOR_TEAM_ABBREVIATION','GAME_TIME']]
df_nuggets_display['game_time_mountain'] = df_nuggets_display['GAME_TIME'].apply(subtract_two_hours)
df_nuggets_display = df_nuggets_display.drop(columns='GAME_TIME')
df_nuggets_display

,GAME_DATE,HOME_TEAM_ABBREVIATION,VISITOR_TEAM_ABBREVIATION,game_time_mountain
0,"MAR 29, 2024",DEN,MIN,07:00 PM
1,"MAR 31, 2024",DEN,CLE,01:30 PM
2,"APR 02, 2024",DEN,SAS,07:00 PM
3,"APR 04, 2024",LAC,DEN,08:00 PM
4,"APR 06, 2024",DEN,ATL,07:00 PM


In [92]:
df_nuggets_display['display_text'] = df_nuggets_display['VISITOR_TEAM_ABBREVIATION'] + ' @ ' + df_nuggets_display['HOME_TEAM_ABBREVIATION'] + ' - ' + df_nuggets_display['game_time_mountain']

In [109]:
df_nuggets_display = df_nuggets_display[['GAME_DATE','display_text']]
df_nuggets_display

,GAME_DATE,display_text
0,"MAR 29, 2024",MIN @ DEN - 07:00 PM
1,"MAR 31, 2024",CLE @ DEN - 01:30 PM
2,"APR 02, 2024",SAS @ DEN - 07:00 PM
3,"APR 04, 2024",DEN @ LAC - 08:00 PM
4,"APR 06, 2024",ATL @ DEN - 07:00 PM


In [118]:
# Create message from the columns
df = df_nuggets_display.copy()
df['display_text'] = df['GAME_DATE'] + ': ' + df['display_text']
df = df[['display_text']]
df.display_text.to_string(index=False)

'MAR 29, 2024: MIN @ DEN - 07:00 PM\nMAR 31, 2024: CLE @ DEN - 01:30 PM\nAPR 02, 2024: SAS @ DEN - 07:00 PM\nAPR 04, 2024: DEN @ LAC - 08:00 PM\nAPR 06, 2024: ATL @ DEN - 07:00 PM'

## Try for playoff games

In [25]:
from nba_api.stats.endpoints import TeamGameLogs
from nba_api.stats.library.parameters import SeasonType, SeasonTypePlayoffs

In [13]:
# Function to get the team ID by team name
def get_team_id(team_name):
    nba_teams = teams.get_teams()
    for team in nba_teams:
        if team['full_name'] == team_name:
            return team['id']
    return None

In [15]:
# Replace 'Los Angeles Lakers' with the name of the desired NBA team
team_name = 'Denver Nuggets'
team_id = get_team_id(team_name)
season = '2023-24'

In [27]:
# Fetch the game logs for the specified team and season
team_game_logs = TeamGameLogs(
    team_id_nullable=team_id,
    season_nullable=season,
    season_type_nullable=SeasonTypePlayoffs.playoffs
)

In [28]:
# Get the game logs as a DataFrame
game_logs_df = team_game_logs.get_data_frames()[0]

In [29]:
game_logs_df

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,AVAILABLE_FLAG


In [30]:
game_logs_df.columns

Index(['SEASON_YEAR', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS',
       'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK',
       'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK',
       'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK',
       'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK',
       'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK', 'AVAILABLE_FLAG'],
      dtype='object')

## Scraping

In [31]:
import requests
from bs4 import BeautifulSoup

In [32]:
# URL of the Denver Nuggets' schedule page
url = 'https://www.nba.com/nuggets/schedule'

In [33]:
# Send a GET request to the URL
response = requests.get(url)

In [35]:
# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the schedule section (this is an example selector, adjust as needed)
    schedule_section = soup.find('section', {'class': 'schedule'})

    # Initialize an empty list to hold the games
    games = []

    # Iterate through the games in the schedule section
    for game in schedule_section.find_all('div', {'class': 'game'}):
        # Extract details of each game (adjust selectors as needed)
        date = game.find('span', {'class': 'date'}).text.strip()
        time = game.find('span', {'class': 'time'}).text.strip()
        opponent = game.find('span', {'class': 'opponent'}).text.strip()
        location = game.find('span', {'class': 'location'}).text.strip()

        # Store the game information in a dictionary
        game_info = {
            'date': date,
            'time': time,
            'opponent': opponent,
            'location': location,
        }

        # Add the game information to the list of games
        games.append(game_info)

    # Print the list of games
    for game in games:
        print(game)
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

AttributeError: 'NoneType' object has no attribute 'find_all'

## thesportsdb